In [1]:
import numpy as np
from scipy.linalg import sqrtm

In [2]:
A = np.array([[1, 2], [3, 4]])
A_inv = np.linalg.inv(A)
A_inv_sqrt = sqrtm(A_inv)

In [3]:
A_inv_sqrt

array([[0.10306396+1.24742804j, 0.1502082 -0.5706074j ],
       [0.22531231-0.8559111j , 0.32837626+0.39151694j]])

In [4]:
A_sqrt = sqrtm(A)
A_sqrt

array([[0.5536886+0.46439415j, 0.8069607-0.21242648j],
       [1.2104411-0.31863973j, 1.7641296+0.14575444j]], dtype=complex64)

In [51]:
import pandas as pd
import numpy as np

df = pd.read_csv('speed_gangnam_0.csv').drop(columns=['Unnamed: 0'])

import pickle

with open('adj_mx_gangnam.pkl', 'rb') as file:
    data = pickle.load(file, encoding='CP949')
adj = data[2]

df.shape, adj.shape

((2880, 506), (506, 506))

### Graph Convolution

In [83]:
image = df.values
kernel = adj

#image = np.random.random((5, 5))
#kernel = np.random.random((3, 3))

image_height, image_width = image.shape[0], image.shape[1]
kernel_height, kernel_width = kernel.shape[0], kernel.shape[1]

padding = 0
strides = 1

output_height = int(((image_height - kernel_height + 2 * padding) / strides) + 1)
output_width = int(((image_width - kernel_width + 2 * padding) / strides) + 1)

output = np.zeros((output_height, output_width))

In [84]:
imagePadded = image

In [85]:
imagePadded.shape

(2880, 506)

In [86]:
for y in range(0, output_height):
    if (y * strides + kernel_height) <= imagePadded.shape[0]:
        for x in range(0, output_width):
            if (x * strides + kernel_width) <= imagePadded.shape[1]:
                #print(y*strides, x*strides, y*strides+kernel_height, x*strides + kernel_width)
                output[y][x] = np.sum(
                    imagePadded[y*strides: y*strides + kernel_height,
                                x*strides: x*strides + kernel_width] 
                    * kernel).astype(np.float32)                

In [87]:
output

array([[92573.203125 ],
       [92522.5390625],
       [92240.7734375],
       ...,
       [80673.4375   ],
       [79964.96875  ],
       [80014.125    ]])

In [173]:
adj

array([[1.        , 0.40327677, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.40327677, 1.        , 0.91133416, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.91133416, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.38495174,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.38495174, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]], dtype=float32)

In [88]:
output.sum()

195880733.90625

In [93]:
np.dot(df, adj)

array([[273.16220448, 202.56419772, 167.72485007, ..., 303.57415218,
        228.84379668, 199.02047791],
       [249.66616005, 189.45193931, 156.64285913, ..., 281.16564069,
        228.37975824, 242.9986063 ],
       [243.86885553, 202.736215  , 185.90254924, ..., 279.41321781,
        241.39661813, 220.37047939],
       ...,
       [235.83328419, 189.56337791, 162.86751901, ..., 282.54185251,
        226.08895569, 219.79172238],
       [233.76795502, 184.6679757 , 160.88231704, ..., 269.77866633,
        218.6070577 , 212.47407996],
       [240.38100303, 195.13222251, 167.55241826, ..., 294.67956451,
        233.72052721, 227.11547766]])

In [91]:
df

,1210007700,1210008500,1210009500,1210010300,1210011300,1210013700,1220011900,1220016300,1220021100,1220025100,...,1210013800,1210013000,1210012200,1210012300,1210013100,1210013900,1210014900,1210015900,1210017100,1210018300
0,32.70,21.00,33.54,43.21,24.31,26.54,35.91,29.00,28.78,21.76,...,24.98,24.54,32.76,43.75,32.36,32.13,35.47,22.96,26.62,14.00
1,31.50,38.00,25.61,43.89,22.75,24.37,30.73,29.07,25.92,22.66,...,26.22,25.14,32.15,43.75,25.00,35.04,26.00,16.00,27.22,29.58
2,22.00,38.00,47.00,54.00,19.00,24.36,35.75,24.63,21.96,22.67,...,21.55,25.16,34.37,43.75,25.00,33.00,36.32,22.42,32.40,11.00
3,31.00,34.39,32.70,44.15,23.64,27.76,40.06,41.00,28.65,21.68,...,25.33,24.11,33.41,28.61,29.88,34.93,37.35,21.35,29.07,32.34
4,39.20,37.65,30.75,43.66,22.76,28.04,28.10,23.25,23.00,19.92,...,24.44,24.15,34.94,28.11,33.88,33.53,39.30,30.68,27.68,30.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,27.50,36.33,29.53,39.86,26.15,20.05,25.37,18.27,21.88,18.25,...,28.17,24.28,38.21,21.95,29.47,28.47,31.52,32.13,28.22,15.96
2876,25.51,37.62,30.48,41.88,26.52,23.00,31.94,16.56,24.29,19.14,...,28.65,24.49,32.52,27.44,31.76,33.00,44.68,24.89,33.04,11.00
2877,29.40,28.26,31.81,39.68,26.45,24.28,28.92,18.00,23.41,21.11,...,32.26,23.88,32.02,26.94,30.75,31.10,17.00,24.70,30.21,18.65
2878,24.00,28.21,29.02,41.93,25.49,25.38,25.57,17.61,23.63,21.70,...,27.08,27.32,34.00,26.27,27.32,29.02,37.90,27.58,30.23,15.41


In [103]:
(df.T[0] * adj[0]).sum() / adj[0].sum()

41.117866130228606

In [106]:
np.where(adj[0] > 0)

(array([  0,   1,  28, 326, 327, 344, 345, 487], dtype=int64),)

In [116]:
df.T[0][np.where(adj[0]>0)[0]].values

array([32.7 , 21.  , 43.2 , 25.31, 56.51, 40.61, 51.97, 46.49])

In [115]:
adj[0][np.where(adj[0]>0)[0]]

array([1.        , 0.40327677, 0.40327677, 0.9191608 , 0.99950624,
       0.99950624, 0.9191608 , 0.99950624], dtype=float32)

In [120]:
v_sum = 0
adj_sum = adj[0].sum()

for idx in np.where(adj[0] > 0)[0]:
    weight = adj[0][idx] / adj_sum
    v = df.T[0][idx] * weight
    v_sum += v
v_sum

41.11786716118455

In [126]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,2870,2871,2872,2873,2874,2875,2876,2877,2878,2879
1210007700,32.70,31.50,22.00,31.00,39.20,37.01,34.00,30.65,25.50,28.30,...,28.00,31.57,23.82,24.00,30.55,27.50,25.51,29.40,24.00,31.77
1210008500,21.00,38.00,38.00,34.39,37.65,36.75,48.00,36.86,48.00,37.21,...,41.00,30.41,33.91,29.00,34.97,36.33,37.62,28.26,28.21,38.30
1210009500,33.54,25.61,47.00,32.70,30.75,31.91,44.00,32.23,38.00,31.82,...,30.00,31.36,28.06,32.00,29.87,29.53,30.48,31.81,29.02,30.65
1210010300,43.21,43.89,54.00,44.15,43.66,45.49,41.05,42.92,49.00,43.37,...,37.58,39.35,40.13,52.00,38.04,39.86,41.88,39.68,41.93,39.34
1210011300,24.31,22.75,19.00,23.64,22.76,23.88,17.00,21.71,18.00,22.48,...,29.16,25.57,25.31,27.00,26.69,26.15,26.52,26.45,25.49,34.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210013900,32.13,35.04,33.00,34.93,33.53,30.09,34.00,35.00,36.00,31.59,...,31.07,41.00,18.00,34.00,29.57,28.47,33.00,31.10,29.02,32.43
1210014900,35.47,26.00,36.32,37.35,39.30,40.02,39.67,30.00,30.00,30.00,...,26.34,14.00,14.00,36.00,26.00,31.52,44.68,17.00,37.90,39.29
1210015900,22.96,16.00,22.42,21.35,30.68,28.42,33.60,33.51,23.00,23.00,...,29.43,27.77,23.99,32.00,24.16,32.13,24.89,24.70,27.58,29.04
1210017100,26.62,27.22,32.40,29.07,27.68,25.25,26.22,23.00,26.54,32.20,...,29.38,25.71,30.08,33.26,27.71,28.22,33.04,30.21,30.23,34.64


In [168]:
outputY = df.T.shape[0]
outputX = df.T.shape[1]
output = np.zeros((outputY, outputX))
output.shape

(506, 2880)

In [169]:
def translate_time(second):
    h = second // 3600
    m = int(second // 60 - h * 60)
    s = round(second % 60, 2)
    
    time = ''
    if h > 0: 
        time += str(h) + 'h '
        time += str(m) + 'm '
    elif m > 0: time += str(m) + 'm '
    time += str(s) + 's'
    
    return time

In [170]:
import time

start = time.time()
print('0 ~ 10', end = ': ')
for y in range(outputY):
    if y % 10 == 0 and y > 0:
        print(translate_time(time.time() - start))
        print(str(y) + ' ~ ' + str(y+10), end = ': ')
        start = time.time()
        
    for x in range(outputX):
        v_sum = 0
        adj_sum = adj[y].sum()
        
        for idx in np.where(adj[y] > 0)[0]:
            weight = adj[y][idx] / adj_sum
            v = df.T[x][idx] * weight
            v_sum += v
        output[y][x] = v_sum
        
print(translate_time(time.time() - start))
output

0 ~ 10: 53.32s
10 ~ 20: 30.59s
20 ~ 30: 53.37s
30 ~ 40: 45.89s
40 ~ 50: 35.75s
50 ~ 60: 41.21s
60 ~ 70: 53.07s
70 ~ 80: 39.39s
80 ~ 90: 54.55s
90 ~ 100: 44.78s
100 ~ 110: 52.73s
110 ~ 120: 54.32s
120 ~ 130: 1m 5.73s
130 ~ 140: 51.38s
140 ~ 150: 34.57s
150 ~ 160: 48.4s
160 ~ 170: 53.51s
170 ~ 180: 45.55s
180 ~ 190: 38.3s
190 ~ 200: 43.86s
200 ~ 210: 1m 2.5s
210 ~ 220: 43.73s
220 ~ 230: 54.58s
230 ~ 240: 56.47s
240 ~ 250: 53.43s
250 ~ 260: 51.64s
260 ~ 270: 52.95s
270 ~ 280: 48.61s
280 ~ 290: 56.37s
290 ~ 300: 55.22s
300 ~ 310: 43.31s
310 ~ 320: 46.24s
320 ~ 330: 33.43s
330 ~ 340: 50.87s
340 ~ 350: 35.24s
350 ~ 360: 40.12s
360 ~ 370: 53.14s
370 ~ 380: 41.46s
380 ~ 390: 1m 1.26s
390 ~ 400: 1m 9.51s
400 ~ 410: 1m 5.6s
410 ~ 420: 1m 2.15s
420 ~ 430: 54.02s
430 ~ 440: 36.55s
440 ~ 450: 40.38s
450 ~ 460: 46.11s
460 ~ 470: 58.42s
470 ~ 480: 37.88s
480 ~ 490: 41.21s
490 ~ 500: 47.89s
500 ~ 510: 34.42s


array([[41.11786716, 37.58111418, 36.70847217, ..., 35.49891413,
        35.18802949, 36.18346159],
       [37.26839051, 34.85595636, 37.30003864, ..., 34.87645924,
        33.97578792, 35.90103252],
       [33.90569951, 31.66546705, 37.58033442, ..., 32.92378652,
        32.52247653, 33.87084227],
       ...,
       [32.1178778 , 29.74707706, 29.56167226, ..., 29.89267902,
        28.54234516, 31.17683823],
       [25.76884023, 25.71658743, 27.18234439, ..., 25.45863277,
        24.61613742, 26.31798204],
       [22.74371553, 27.76945981, 25.18355673, ..., 25.11741732,
        24.28116985, 25.95436342]])

In [171]:
df.values.mean(), df.values.std()

(28.274181001592034, 11.135170919736774)

In [172]:
output.mean(), output.std()

(28.27640308928024, 7.405970114851601)

In [127]:
df.T[1][0]

31.5

In [128]:
for i in range(adj.shape[0]):
    v_sum = 0
    adj_sum = adj[i].sum()
    
    for idx in np.where(adj[i] > 0)[0]:
        weight = adj[i][idx] / adj_sum
        v = df.T[i][idx] * weight
        v_sum += v
    print(v_sum, df.T[i][0])

41.11786716118455 32.7
34.85595635861158 31.5
37.58033442273736 22.0
28.615117293447256 31.0
27.618520742580298 39.2
27.215403730664406 37.01
36.34479475690052 34.0
28.26424498785287 30.65
25.86498176276684 25.5
25.88601643245667 28.3
25.366655357703564 32.0
30.84996803790331 29.5
30.77785994250327 28.0
58.201504267156125 25.86
48.78189110025763 30.0
52.717776196599004 35.21
36.65018258616328 34.5
31.12402012761682 36.5
29.33694441154599 33.0
25.99127701342106 27.0
27.681830271855 38.65
25.974910844974218 34.0
30.304181253779674 38.0
31.87220797846094 34.98
28.996861449535935 24.5
29.26037752301432 26.5
30.791403799653054 29.0
36.593428273499015 29.0
45.196770541369915 61.73
27.5763127348572 41.89
32.349721771031625 46.0
32.453540017902846 41.0
31.03853891339153 37.5
30.16751674428582 37.5
33.22747668832541 35.68
35.71110929250717 38.8
36.882149654366074 39.69
33.8626780279167 33.5
30.515804379321633 33.5
30.2249246719107 33.0
27.770208964347837 33.1
30.555431426912545 31.5
31.21103336

42.41087309941649 21.0
36.30381160970777 21.0
27.330175332576037 39.21
34.450737634859976 31.5
31.829774799458683 31.5
31.755381691325457 31.5
29.819821858927604 32.0
34.20602823976428 6.0
37.67841061655432 28.65
30.567371831387284 32.0
28.144155061617496 37.5
46.446318041794 40.0
51.97686668038368 40.0
86.25 25.47
57.319375051259996 45.6
45.34307805590331 40.5
33.32968707695604 32.0
31.010638925135133 28.6
34.18682405143976 30.41
33.68685548007488 34.5
29.06881784442812 29.0
27.65378693282604 27.5
30.880512241311376 28.5
31.015772490203382 34.5
32.02605924692004 32.0
35.03451041487977 51.88
39.65910044774413 28.0
23.172572485953566 25.5
22.349739808104932 26.0
31.739679074622686 25.5
31.046889116689563 22.5
28.947822928205134 32.5
25.923507617861034 27.0
26.542828300576655 31.0
26.572220563292504 28.0
27.28124477364123 29.5
26.266427534222604 36.0
26.236765370741484 36.0
31.870430970229208 34.0
31.166715818122025 24.5
28.333135342262683 27.5
38.781636914163826 15.3
38.0107365822047 31

In [95]:
adj[0]

array([1.        , 0.40327677, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.40327677, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [102]:
adj[0].sum()

6.6433945

In [5]:
import torch.nn as nn
import numpy as np
from abc import abstractmethod

In [2]:
class BaseModel(nn.Module):
    @abstractmethod
    def forward(self, *inputs):
        raise NotImplementedError

    def __str__(self):
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return super().__str__() + '\nTrainable parameters: {}'.format(params)

In [ ]:
Degree = np.zeros((inputX, inputY))
for i in range(len(adj)):
    deg = 0
    for j in range(len(adj)):
        deg += adj[i][j]
    Degree[i] = deg

Laplacian = Degree - Adj
Unit = np.zeros((inputX, inputY))
for i in range(len(Degree)):
    

Laplacian_Normalized = np.dot(np.dot(np.inverse(np.sqrt(D)), adj), np.inverse(np.sqrt(D)))+

In [ ]:
class Chebyshev:
    def __init__(self, a, b, n, func): 
        self.a = a
        self.b = b
        self.func = func

        bma = 0.5 * (b - a)
        bpa = 0.5 * (b + a)
        f = [func(math.cos(math.pi * (k + 0.5) / n) * bma + bpa) for k in range(n)]
        fac = 2.0 / n
        self.c = [fac * sum([f[k] * math.cos(math.pi * j * (k + 0.5) / n)
                  for k in range(n)]) for j in range(n)]

    def eval(self, x):
        a,b = self.a, self.b
        assert(a <= x <= b)
        y = (2.0 * x - a - b) * (1.0 / (b - a))
        y2 = 2.0 * y
        (d, dd) = (self.c[-1], 0)
        for cj in self.c[-2:0:-1]:
            (d, dd) = (y2 * d - dd + cj, d)
        return y * d - dd + 0.5 * self.c[0]

In [3]:
class DiffusionGraphConv(BaseModel):
    def __init__(self, supports, input_dim, hid_dim, num_nodes, max_diffusion_step, output_dim, bias_start=0.0):
        super(DiffusionGraphConv, self).__init__()
        self.num_matrices = len(supports) * max_diffusion_step + 1
        input_size = input_dim + hid_dim
        self._num_nodes = num_nodes
        self._max_diffusion_step = max_diffusion_step
        self._supports = supports
        self.weight = nn.Parameter(torch.FloatTensor(size=(input_size*self.num_matrices, output_dim)))
        self.biases = nn.Parameter(torch.FloatTensor(size=(output_dim,)))
        nn.init.xavier_normal_(self.weight.data, gain=1.414)
        nn.init.constant_(self.biases.data, val=bias_start)

    @staticmethod
    def _concat(x, x_):
        x_ = torch.unsqueeze(x_, 0)
        return torch.cat([x, x_], dim=0)

    def forward(self, inputs, state, output_size, bias_start=0.0):
        batch_size = inputs.shape[0]
        inputs = torch.reshape(inputs, (batch_size, self._num_nodes, -1))
        state = torch.reshape(state, (batch_size, self._num_nodes, -1))
        inputs_and_state = torch.cat([inputs, state], dim=2)
        input_size = inputs_and_state.shape[2]

        x = inputs_and_state
        x0 = torch.transpose(x, dim0=0, dim1=1)
        x0 = torch.transpose(x0, dim0=1, dim1=2)
        x0 = torch.reshape(x0, shape=[self._num_nodes, input_size * batch_size])
        x = torch.unsqueeze(x0, dim=0)

        if self._max_diffusion_step == 0:
            pass
        else:
            for support in self._supports:
                x1 = torch.sparse.mm(support, x0)
                x = self._concat(x, x1)
                for k in range(2, self._max_diffusion_step + 1):
                    x2 = 2 * torch.sparse.mm(support, x1) - x0
                    x = self._concat(x, x2)
                    x1, x0 = x2, x1

        x = torch.reshape(x, shape=[self.num_matrices, self._num_nodes, input_size, batch_size])
        x = torch.transpose(x, dim0=0, dim1=3)
        x = torch.reshape(x, shape=[batch_size * self._num_nodes, input_size * self.num_matrices])

        x = torch.matmul(x, self.weight)
        x = torch.add(x, self.biases)
        
        return torch.reshape(x, [batch_size, self._num_nodes * output_size])

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionLayer(nn.Module):
    def __init__(self, in_channels, num_of_vertices, num_of_features):
        super(AttentionLayer, self).__init__()

        self.W1 = nn.Parameter(torch.FloatTensor(num_of_features)) # (T)
        self.W2 = nn.Parameter(torch.FloatTensor(in_channels, num_of_features)) # (F, T)
        self.W3 = nn.Parameter(torch.FloatTensor(in_channels)) # (F)

        self.Vs = nn.Parameter(torch.FloatTensor(num_of_vertices, num_of_vertices)) # (N, N)
        self.bs = nn.Parameter(torch.FloatTensor(1, num_of_vertices, num_of_vertices)) # (1, N, N)

    def forward(self, x): #X: (B, N, F, T)
        xw = torch.matmul(x, self.W1) # (B, N, F, T) (T) -> (B, N, F)
        txw = torch.matmul(self.W3, x).transpose(-1, -2) # (F) (B, N, F, T) -> (B, N, T) -> (B, T, N)
        product = torch.matmul(torch.matmul(xw, self.W2), txw) # (B, N, F) (F, T) -> (B, N, T) (B, T, N) -> (B, N, N)

        sig = torch.sigmoid(product + self.bs) 
        S = torch.matmul(self.Vs, sig) # (N, N) (B, N, N) -> (B, N, N)
        S_prime = F.softmax(S, dim=1)
        
        return S_prime

In [2]:
import pandas as pd

df = pd.read_csv('speed_gangnam_0.csv').drop(columns=['Unnamed: 0'])
df

,1210007700,1210008500,1210009500,1210010300,1210011300,1210013700,1220011900,1220016300,1220021100,1220025100,...,1210013800,1210013000,1210012200,1210012300,1210013100,1210013900,1210014900,1210015900,1210017100,1210018300
0,32.70,21.00,33.54,43.21,24.31,26.54,35.91,29.00,28.78,21.76,...,24.98,24.54,32.76,43.75,32.36,32.13,35.47,22.96,26.62,14.00
1,31.50,38.00,25.61,43.89,22.75,24.37,30.73,29.07,25.92,22.66,...,26.22,25.14,32.15,43.75,25.00,35.04,26.00,16.00,27.22,29.58
2,22.00,38.00,47.00,54.00,19.00,24.36,35.75,24.63,21.96,22.67,...,21.55,25.16,34.37,43.75,25.00,33.00,36.32,22.42,32.40,11.00
3,31.00,34.39,32.70,44.15,23.64,27.76,40.06,41.00,28.65,21.68,...,25.33,24.11,33.41,28.61,29.88,34.93,37.35,21.35,29.07,32.34
4,39.20,37.65,30.75,43.66,22.76,28.04,28.10,23.25,23.00,19.92,...,24.44,24.15,34.94,28.11,33.88,33.53,39.30,30.68,27.68,30.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,27.50,36.33,29.53,39.86,26.15,20.05,25.37,18.27,21.88,18.25,...,28.17,24.28,38.21,21.95,29.47,28.47,31.52,32.13,28.22,15.96
2876,25.51,37.62,30.48,41.88,26.52,23.00,31.94,16.56,24.29,19.14,...,28.65,24.49,32.52,27.44,31.76,33.00,44.68,24.89,33.04,11.00
2877,29.40,28.26,31.81,39.68,26.45,24.28,28.92,18.00,23.41,21.11,...,32.26,23.88,32.02,26.94,30.75,31.10,17.00,24.70,30.21,18.65
2878,24.00,28.21,29.02,41.93,25.49,25.38,25.57,17.61,23.63,21.70,...,27.08,27.32,34.00,26.27,27.32,29.02,37.90,27.58,30.23,15.41


In [10]:
adj = np.ones((506, 506))
adj.shape

(506, 506)

In [12]:
import torch
import torch.nn as nn

in_features = 506
out_features = 506

W = nn.Parameter(torch.empty(size=(in_features, out_features)))
a = nn.Parameter(torch.empty(size=(2*out_features, 1)))

nn.init.xavier_uniform_(W.data, gain=1.414)
nn.init.xavier_uniform_(a.data, gain=1.414)

tensor([[ 0.0782],
        [-0.0455],
        [-0.0232],
        ...,
        [-0.0206],
        [-0.0935],
        [-0.0994]])

In [13]:
leakyrelu = nn.LeakyReLU(0.7)

In [17]:
h = df.values
h = torch.FloatTensor(h)
Wh = torch.mm(h, W)

In [18]:
Wh.shape

torch.Size([2880, 506])

In [27]:
a[:out_features, :].shape

torch.Size([506, 1])

In [28]:
Wh1 = torch.matmul(Wh, a[:out_features, :])
Wh2 = torch.matmul(Wh, a[out_features:, :])

In [29]:
e = Wh1 + Wh2.T
e = leakyrelu(e)
e.shape

torch.Size([2880, 2880])

In [30]:
adj = torch.FloatTensor(adj)

In [31]:
adj.shape

torch.Size([506, 506])

In [33]:
zero_vec = -9e15 * torch.ones_like(e)
attention = torch.where(adj > 0, e, zero_vec)

RuntimeError: The size of tensor a (506) must match the size of tensor b (2880) at non-singleton dimension 1

In [34]:
h_prime = torch.matmul(zero_vec, Wh)

In [35]:
h_prime.shape

torch.Size([2880, 506])

In [36]:
h_prime

tensor([[ 1.2578e+21,  5.1637e+19, -8.4420e+20,  ...,  2.5787e+20,
          7.2114e+20,  2.9497e+20],
        [ 1.2578e+21,  5.1637e+19, -8.4420e+20,  ...,  2.5787e+20,
          7.2114e+20,  2.9497e+20],
        [ 1.2578e+21,  5.1637e+19, -8.4420e+20,  ...,  2.5787e+20,
          7.2114e+20,  2.9497e+20],
        ...,
        [ 1.2578e+21,  5.1637e+19, -8.4420e+20,  ...,  2.5787e+20,
          7.2114e+20,  2.9497e+20],
        [ 1.2578e+21,  5.1637e+19, -8.4420e+20,  ...,  2.5787e+20,
          7.2114e+20,  2.9497e+20],
        [ 1.2578e+21,  5.1637e+19, -8.4420e+20,  ...,  2.5787e+20,
          7.2114e+20,  2.9497e+20]], grad_fn=<MmBackward0>)

In [37]:
h

tensor([[32.7000, 21.0000, 33.5400,  ..., 22.9600, 26.6200, 14.0000],
        [31.5000, 38.0000, 25.6100,  ..., 16.0000, 27.2200, 29.5800],
        [22.0000, 38.0000, 47.0000,  ..., 22.4200, 32.4000, 11.0000],
        ...,
        [29.4000, 28.2600, 31.8100,  ..., 24.7000, 30.2100, 18.6500],
        [24.0000, 28.2100, 29.0200,  ..., 27.5800, 30.2300, 15.4100],
        [31.7700, 38.3000, 30.6500,  ..., 29.0400, 34.6400, 15.3000]])

In [6]:
traffic = df.values

In [8]:
in_channels = 207
num_of_features = 2880
num_of_vertices = 207

In [40]:
W1 = nn.Parameter(torch.FloatTensor(num_of_features)) # (T)
W2 = nn.Parameter(torch.FloatTensor(num_of_features)) # (T)

Vs = nn.Parameter(torch.FloatTensor(num_of_vertices, num_of_vertices)) # (N, N)
bs = nn.Parameter(torch.FloatTensor(1, num_of_vertices, num_of_vertices)) 

In [11]:
W1.shape, W2.shape, W3.shape, Vs.shape, bs.shape

(torch.Size([2880]),
 torch.Size([207, 2880]),
 torch.Size([207]),
 torch.Size([207, 207]),
 torch.Size([1, 207, 207]))

In [23]:
x = torch.FloatTensor(traffic)
x = torch.transpose(x, -1, -2)
x.shape

torch.Size([506, 2880])

In [39]:
x_prime = x.repeat(16, 1).reshape(-1, 506, 2880)
x_prime.shape

torch.Size([16, 506, 2880])

In [43]:
xw = torch.matmul(x_prime, W1)
xw.shape

torch.Size([16, 506])

In [44]:
xw = torch.matmul(xw, W2)
xw.shape

RuntimeError: size mismatch, got 16, 16x506,2880

In [ ]:
xw = torch.matmul(x, self.W1) # (B, N, F, T) (T) -> (B, N, F)
        txw = torch.matmul(self.W3, x).transpose(-1, -2) # (F) (B, N, F, T) -> (B, N, T) -> (B, T, N)
        product = torch.matmul(torch.matmul(xw, self.W2), txw) # (B, N, F) (F, T) -> (B, N, T) (B, T, N) -> (B, N, N)

        sig = torch.sigmoid(product + self.bs) 
        S = torch.matmul(self.Vs, sig) # (N, N) (B, N, N) -> (B, N, N)
        S_prime = F.softmax(S, dim=1)

In [38]:
in_features = 506
out_features = 506

alpha = 0.5

W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
nn.init.xavier_normal_(W.data, gain=1.414)

a = nn.Parameter(torch.zeros(size=(1, 2*out_features)))
nn.init.xavier_normal_(a.data, gain=1.414)

tensor([[ 0.0488,  0.0575, -0.0026,  ...,  0.0157, -0.1635,  0.0107]])

In [39]:
h = torch.FloatTensor(df.values)
N = h.size()[0]
N

2880

In [42]:
adj = np.ones((506, 506))
adj = torch.FloatTensor(adj)
edge = adj.nonzero().t()

In [43]:
h = torch.mm(h, W)

In [44]:
h.shape

torch.Size([2880, 506])

In [45]:
edge_h = torch.cat((h[edge[0, :], :], h[edge[1, :], :]), dim=1).t()

In [48]:
leakyrelu = nn.LeakyReLU(alpha)
edge_e = torch.exp(-leakyrelu(a.mm(edge_h).squeeze()))

In [49]:
edge.shape, edge_e.shape

(torch.Size([2, 256036]), torch.Size([256036]))

In [52]:
class SpecialSpmmFunction(torch.autograd.Function):
    """Special function for only sparse region backpropataion layer."""
    @staticmethod
    def forward(ctx, indices, values, shape, b):
        assert indices.requires_grad == False
        a = torch.sparse_coo_tensor(indices, values, shape)
        ctx.save_for_backward(a, b)
        ctx.N = shape[0]
        return torch.matmul(a, b)

    @staticmethod
    def backward(ctx, grad_output):
        a, b = ctx.saved_tensors
        grad_values = grad_b = None
        if ctx.needs_input_grad[1]:
            grad_a_dense = grad_output.matmul(b.t())
            edge_idx = a._indices()[0, :] * ctx.N + a._indices()[1, :]
            grad_values = grad_a_dense.view(-1)[edge_idx]
        if ctx.needs_input_grad[3]:
            grad_b = a.t().matmul(grad_output)
        return None, grad_values, None, grad_b


class SpecialSpmm(nn.Module):
    def forward(self, indices, values, shape, b):
        return SpecialSpmmFunction.apply(indices, values, shape, b)

In [53]:
special_spmm = SpecialSpmm()

In [54]:
e_rowsum = special_spmm(edge, edge_e, torch.Size([N,N]), torch.ones(size=(N,1)))

In [55]:
e_rowsum

tensor([[ 959.4462],
        [ 286.5499],
        [1987.9010],
        ...,
        [   0.0000],
        [   0.0000],
        [   0.0000]], grad_fn=<SpecialSpmmFunctionBackward>)

In [56]:
h_prime = special_spmm(edge, edge_e, torch.Size([N,N]), h)
h_prime.shape

torch.Size([2880, 506])

In [57]:
h_prime = h_prime.div(e_rowsum)
h_prime

tensor([[-20.7039, -22.8317,  45.6013,  ..., -18.8642, -19.1760, -36.4217],
        [-20.7096, -22.8344,  45.6733,  ..., -18.9175, -19.1660, -36.4120],
        [-20.7017, -22.8398,  45.5797,  ..., -18.8467, -19.1660, -36.4197],
        ...,
        [     nan,      nan,      nan,  ...,      nan,      nan,      nan],
        [     nan,      nan,      nan,  ...,      nan,      nan,      nan],
        [     nan,      nan,      nan,  ...,      nan,      nan,      nan]],
       grad_fn=<DivBackward0>)

In [60]:
h_prime.isnan().sum()

tensor(1201244)

In [61]:
h_prime.shape

torch.Size([2880, 506])

In [62]:
h_prime.shape[0] * h_prime.shape[1]

1457280

In [64]:
1201244 / 1457280

0.8243055555555555